# Treinando modelo de regressão logística com SDK do Azure para Python

## Verificar o pacote **azure-ai-ml**

É necessário a última versão dos pacotes **azure-ai-ml** e **mlflow** para rodar o código nesse notebook. Executar as células abaixo para verificar se está na última versão.

> **Obs**:
> Se os pacotes **azure-ai-ml** e **mlflow** não estiver instalado, execute `pip install azure-ai-ml` e `pip install mlflow azureml-mlflow` para instalar.

In [52]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.26.5
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [56]:
pip show mlflow azureml-mlflow

Name: mlflow
Version: 2.22.0
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: 
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.
        
                                        Apache License
                                   Version 2.0, January 2004
                                http://www.apache.org/licenses/
        
           TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION
        
           1. Definitions.
        
              "License" shall mean the terms and conditions for use, reproduction,
              and distribution as defined by Sections 1 through 9 of this document.
        
              "Licensor" shall mean the copyright owner or entity authorized by
              the copyright owner that is granting the License.
        
              "Legal Entity" shall mean the union of the acting entity and all
              other entities that control, are controlled by, 

## Conectar workspace do Azure Machine Learning

Para conectar a workspace é preciso ter antes criado os seguintes itens: Resource Group > Workspace Azure Machine Learning > Instância de Computação e Cluster.

No script será necessário identificar os parâmetros, mas ao executar esse notebook dentro de um Studio do Azure Machine Learning, esses podem ser setados como Default.

In [26]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Verifica se consegue gerar token
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Em caso de falha, o Browser pedirá as credenciais.
    credential = InteractiveBrowserCredential()

In [27]:
# Config MLClient
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


## Treinando o Modelo

Para treinar um modelo, é preciso criar o script python (**vendas-sorvetes.py**) na pasta **src**. O script utilizará o arquivo **vendas_sorvetes_clima_itanhaem_2023.csv** da mesma pasta.

In [53]:
%%writefile src/vendas-sorvetes-2.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Dados
df = pd.read_csv('vendas_sorvetes_clima_itanhaem_2023.csv')
df['ocorrencia_chuva'] = df['ocorrencia_chuva'].map({'Sim': 1, 'Não': 0})
X, y = df[['temp_media', 'precipitacao', 'ocorrencia_chuva']].values, df['Qtd'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Modelo
model = LinearRegression().fit(X_train, y_train)
y_hat = model.predict(X_test)

# Métricas
mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)

mlflow.log_metric("MAE", mae)
mlflow.log_metric("MSE", mse)
mlflow.log_metric("RMSE", rmse)
mlflow.log_metric("R2", r2)

# Gráfico
residuos = y_test - y_hat
plt.scatter(y_hat, residuos)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predições')
plt.ylabel('Resíduos')
plt.title('Gráfico de Resíduos')

os.makedirs("outputs", exist_ok=True)
plt.savefig("outputs/residuos.png")
mlflow.log_artifact("outputs/residuos.png", artifact_path="figuras")



Overwriting src/vendas-sorvetes-2.py


In [36]:
%%writefile src/vendas-sorvetes.py
# importar bibliotecas
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load dos dados de vendas
print("Loading Data...")
df = pd.read_csv('vendas_sorvetes_clima_itanhaem_2023.csv')

# transformando variavel categorica em binária
df['ocorrencia_chuva'] = df['ocorrencia_chuva'].map({'Sim': 1, 'Não': 0})

# separar features e labels
X, y = df[['temp_media','precipitacao','ocorrencia_chuva']].values, df['Qtd'].values

# split dos dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# configurar hyperparâmetro de regularização
reg = 0.01


# treinar modelo de regressão logística
print('Treinando modelo de regressão logística com regularização = ', reg)
model = LinearRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calcular acurácia
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)



# calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))





Overwriting src/vendas-sorvetes.py


Executar a célula abaixo para gerar um job de execução do script:

In [54]:
from azure.ai.ml import command

# configurar o job
job = command(
    code="./src",
    command="python vendas-sorvetes-2.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="computeml",
    display_name="sorvetes-pythonv2-treino",
    experiment_name="vendas-sorvetes-reglinear"
)

# criar job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (0.01 MBs): 100%|██████████| 12858/12858 [00:00<00:00, 464148.66it/s]




Monitor your job at https://ml.azure.com/runs/eager_zoo_qbkgz69tvp?wsid=/subscriptions/f33fd907-1df0-4fcf-b8df-9759ee181952/resourcegroups/rg-project1/workspaces/ws-dio&tid=de26de07-6fc8-4380-b046-12854fc8cd3c
